In [ ]:
import pandas as pd
import json
import pandas as pd
import re
import hashlib
import time
import os
from tqdm import tqdm
import re
import sys
import text_to_audio

In [ ]:
kanji_with_kana_pattern = re.compile(r"\s*(?P<kanji>[\u4E00-\u9FFF]) (?P<kana>[\u3040-\u309F]+)\s*")


def transform_content_to_ruby_rich_text(content):
    return kanji_with_kana_pattern.sub(r"<ruby>\1<rt>\2</rt></ruby>", content)


def split_kanji_and_its_kana(content):
    return kanji_with_kana_pattern.sub(r"\1", content), kanji_with_kana_pattern.sub(r"\2", content)


In [ ]:
def get_md5(content):
    content = content.encode("utf-8")
    md5_hash = hashlib.md5()
    md5_hash.update(content)
    return md5_hash.hexdigest()


In [ ]:
import pandas as pd
import re

sentence_content_pattern = re.compile(r'([\u2460-\u2470])\s*')
invalid_line_pattern = re.compile(r'^[\d\s]+$')

example_sentences = []
groups = []
with open("CS_Word_20240405_22.25.17.txt", "r", encoding="utf-8") as fp:
    matched = False
    sentence = None
    last_line = ""
    for line in fp.readlines():
        line = line.strip()
        if len(line) < 3 or invalid_line_pattern.match(line.strip()):
            continue

        if matched == True and last_line[-1] != "。":
            sentence["content"] += line
            last_line = line
            continue

        if sentence_content_pattern.match(line.strip()):
            matched = True
            sentence = {"content": sentence_content_pattern.sub(r"\1 ", line)}
        else:
            if matched:
                sentence["hiragana"] = ""
                sentence["meaning"] = line
                if sentence["meaning"][1] == " ":
                    sentence["meaning"] = sentence["meaning"][2:]

                if sentence["meaning"][0] == "囉":
                    sentence["meaning"] = sentence["meaning"][1:]

                if sentence["content"][0] == "①" and groups:
                    example_sentences.append(groups)
                    groups = []

                groups.append(sentence)
            matched = False
        last_line = line

if groups:
    example_sentences.append(groups)

print(example_sentences[1])
with open("example_sentences.json", "w", encoding="utf-8") as fp:
    fp.write(json.dumps(example_sentences, ensure_ascii=False, indent=4))
    # pd.DataFrame(example_sentences, dtype=str).fillna("").to_json(fp, orient="records", force_ascii=False, indent=4)

In [ ]:
import json
import pandas as pd

df = pd.read_json("grammar_pd.json", dtype=str)
with open("grammar.json", "r", encoding="utf-8") as f:
    data = json.loads(f.read())
    df = pd.DataFrame(data)
df.head()
detail = df.loc[0].to_dict()
print(type(detail["example"]))
print(detail["example"])

In [ ]:
from grammar_enumeration import grammars, df_grammars

In [ ]:
df_grammars.shape

In [ ]:
df_grammars.hiragana.str.replace(r"[（）()\s()]", "").str.contains(r'なしに')[23]

In [ ]:
keyword = "なくして(は)"
df = df_grammars
df[df.content.str.replace(r"[（）()\s()]", "").str.contains(keyword)
                    | df.hiragana.str.replace(r"[（）()\s()]", "").str.contains(keyword)
                    | df.chinese_meaning.str.contains(keyword) | (df.source == keyword)]

In [ ]:
"其中N1级别201个，N2级别150个，N3级别140个，N4级别130个，N5级别120个。"

import json
import re

level_pattern = re.compile(r"^N([1-5])文法")
unit_pattern = re.compile(r"^第(\d+)单元(.*)")
grammar_start_pattern = re.compile(r"^\d+\. ")
seq_pattern = re.compile(r"^（\d+）")
from collections import defaultdict

lines = []
part_keywords = {'接続': 391, '説明': 401, '例文': 910, '注意': 774, '接续': 308, '说明': 515, '读法': 4, '例词': 2, '补充': 1}
level = 0
current_grammar = ""
current_part = ""
current_seq = ""
output = False
# with open("日语蓝宝书.txt", "r", encoding="utf-8") as fp:
# for idx, line in enumerate(fp.readlines()):
content = """



89. ～とて


（1）

接続

名＋とて

説明

表示假定条件的逆接，“即使……也不例外”“即使是……也……”“甚至……”“就连……”。

例文

△常に冷静な彼とて やはり人間だから、感情的になってしまうこともあるのだろう。 【2009年12月真题】/尽管他平时很冷静，但是也有情绪化的时候吧 。

△最近の電気製品は機能が多すぎる。開発者たちとて すべての機能が必要とは思わないのではないか。 【2007年真题】/最近的电器功能实在太多，就算是开发商也未必会认为所有的功能都有必要吧 。

△私とて 試合に負けたことに悔しい。 /输掉了比赛，我也很懊恼 。

注意

①前面主要接续人名。

②意思与「～としても」 相同，但「～とて」 是比较生硬的表达方式。

（2）

接続

名＋だ/動た形＋とて

説明

表示假定条件的逆接，无论前项怎样，后项的原则都是一样的。“即使……也……”“即便……也……”“就算……也……”。

例文

△たとえ病気だとて 試験に欠席してはいけない。 /就算生病了也不能缺考 。

△いくら 頼 たの んだとて 、できないことはできない。 /再怎么拜托，不能做的事情就是不能做 。

△どんなに後悔したとて 、過ぎたことは今さらどうしようもない。 /就算再怎么后悔，对于已经过去的事情也没有办法 。

注意

常与「たとえ」 「どんなに」 「いくら」 等词一起使用。




"""
grammars = {}
for _ in range(1):
    for idx, line in enumerate(content.split("\n")):
        line = line.strip()
        if not line:
            continue

        if res := level_pattern.findall(line):
            level = int(res[0])
            # print(f"Current Level: {level}")
            continue

        if res := unit_pattern.findall(line):
            # print(f"Current Unit: {res[0][0]}.{res[0][1]}")
            continue

        if grammar_start_pattern.match(line):
            print(f"Current Grammar: {line}")
            current_grammar = line
            grammars[current_grammar] = {}
            # current_part = ""
            # current_seq = ""
            continue

        if seq_pattern.match(line):
            # print(f"Current Seq: {line}")
            current_seq = line
            continue

        lines.append(line)

        if len(line) < 5 and line[:2] in part_keywords:
            current_part = line[:2]
            if current_part == "例文":
                grammars[current_grammar][current_part] = []
            else:
                grammars[current_grammar][current_part] = ""
            continue

        if current_part == "例文":
            grammars[current_grammar][current_part].append(line)
        else:
            grammars[current_grammar][current_part] += line + "\n"

print(json.dumps(grammars, ensure_ascii=False, indent=4))

In [ ]:
"其中N1级别201个，N2级别150个，N3级别140个，N4级别130个，N5级别120个。"

import json
import re

content = """



89. ～とて


（1）

接続

名＋とて

説明

表示假定条件的逆接，“即使……也不例外”“即使是……也……”“甚至……”“就连……”。

例文

△常に冷静な彼とて やはり人間だから、感情的になってしまうこともあるのだろう。 【2009年12月真题】/尽管他平时很冷静，但是也有情绪化的时候吧 。

△最近の電気製品は機能が多すぎる。開発者たちとて すべての機能が必要とは思わないのではないか。 【2007年真题】/最近的电器功能实在太多，就算是开发商也未必会认为所有的功能都有必要吧 。

△私とて 試合に負けたことに悔しい。 /输掉了比赛，我也很懊恼 。

注意

①前面主要接续人名。

②意思与「～としても」 相同，但「～とて」 是比较生硬的表达方式。

（2）

接続

名＋だ/動た形＋とて

説明

表示假定条件的逆接，无论前项怎样，后项的原则都是一样的。“即使……也……”“即便……也……”“就算……也……”。

例文

△たとえ病気だとて 試験に欠席してはいけない。 /就算生病了也不能缺考 。

△いくら 頼 たの んだとて 、できないことはできない。 /再怎么拜托，不能做的事情就是不能做 。

△どんなに後悔したとて 、過ぎたことは今さらどうしようもない。 /就算再怎么后悔，对于已经过去的事情也没有办法 。

注意

常与「たとえ」 「どんなに」 「いくら」 等词一起使用。




"""

level_pattern = re.compile(r"^N([1-5])文法")
unit_pattern = re.compile(r"^第(\d+)单元(.*)")
grammar_start_pattern = re.compile(r"^\d+\. ")
seq_pattern = re.compile(r"^（\d+）")


def get_japanese_sequence_sign(number):
    number = int(number)
    if number <= 0:
        return "⓪"

    return chr(ord("\u2460") + number - 1)


def extract_grammars(content):
    grammars = {}
    lines = []
    part_keywords = {'接続': 391, '説明': 401, '例文': 910, '注意': 774, '接续': 308, '说明': 515, '读法': 4, '例词': 2, '补充': 1}
    keyword_mapping = {
        '接続': "接续",
        '説明': "说明",
        '读法': "说明",
        '例词': "说明",
        '补充': "注意",
    }
    level = 0
    current_grammar = ""
    current_part = ""
    current_seq = ""
    for idx, line in enumerate(content.split("\n")):
        line = line.strip()
        if not line:
            continue

        line = line.replace("真题】", "JLPT】")
        if res := level_pattern.findall(line):
            level = int(res[0])
            # print(f"Current Level: {level}")
            continue

        if res := unit_pattern.findall(line):
            # print(f"Current Unit: {res[0][0]}.{res[0][1]}")
            continue

        if grammar_start_pattern.match(line):
            # print(f"Current Grammar: {line}")
            current_grammar = line
            grammars[current_grammar] = {}
            grammars[current_grammar]["level"] = level
            # current_part = ""
            # current_seq = ""
            continue

        if seq_pattern.match(line):
            print(f"Current Seq: {line}")
            current_seq = line
            continue

        lines.append(line)

        if len(line) < 5 and line[:2] in part_keywords:
            current_part = line[:2]
            current_part = keyword_mapping.get(current_part, current_part)
            index = f"{get_japanese_sequence_sign(line[2])} " if len(line) > 2 else ""
            if current_part not in grammars[current_grammar]:
                if current_part == "例文":
                    grammars[current_grammar][current_part] = []
                else:
                    grammars[current_grammar][current_part] = ""
            continue

        try:
            if current_part == "例文":
                grammars[current_grammar][current_part].append(line)
            else:
                grammars[current_grammar][current_part] += index + line + "\n"
                index = ""
        except Exception:
            print((idx, line))
            raise

    return grammars


In [ ]:
json.dumps("⓪①②③")

In [ ]:
chr(ord("\u2460"))

In [ ]:
kanji_with_kana_pattern = re.compile(r"(?P<kanji>[\u4E00-\u9FFF]) (?P<kana>[\u3040-\u309F]+) ")
content = "中 なか 川 がわ 先 せん 生 せい 「あ、 山 やま 口 ぐち さん。 偶 ぐう 然 ぜん ですね。」"
content = "辞书形 ⇒ 尊他语(辞书形)"
content = "做 | する ⇒ なさる"
content = "84. ～ 中 ちゅう/じゅう"
print(kanji_with_kana_pattern.findall(content))
kanji_with_kana_pattern.sub(r"<ruby>\1<rt>\2</rt></ruby>", content)

In [ ]:


content = "△お 客 きゃく 様 さま 、このお 皿 さら をさげてもよろしいでしょうか 。 /客人您好，这个盘子我可以撤下去了吗 ？"
print(transform_content_to_ruby_rich_text(content))


print(split_kanji_and_its_kana(content))

In [ ]:
def merge_example_sentences(sentences):
    new_sentences = []
    for sentence in sentences:
        if not sentence.startswith("△"):
            new_sentences[-1] += "\n" + sentence
        elif sentence.startswith("△B"):
            new_sentences[-1] += "\n  " + sentence[1:]
        else:
            new_sentences.append(sentence)

    return new_sentences


with open("日语蓝宝书.txt", "r", encoding="utf-8") as fp:
    content = fp.read()

grammars = extract_grammars(content)
# print(json.dumps(grammars, ensure_ascii=False, indent=4))
part_keywords = {'接続': 391, '説明': 401, '例文': 910, '注意': 774, '接续': 308, '说明': 515, '读法': 4, '例词': 2, '补充': 1}
example_sentence_pattern = re.compile(r"^([^/]+?)(【\d+年(?:\d+月)?JLPT】)?/(.*)$", re.DOTALL)
header_pattern = re.compile(r"^△\s*")
formatted_grammars = []
for idx, (grammar, detail) in enumerate(grammars.items()):
    try:
        content = grammar_start_pattern.sub("", grammar)
        hiragana = ""
        rich_text = transform_content_to_ruby_rich_text(content)
        if rich_text != content:
            content, hiragana = split_kanji_and_its_kana(content)

        example_sentences = detail.get("例文") or []
        example_sentences = merge_example_sentences(example_sentences)
        formated_exmpale_sentences = []

        for sentence_seq, sentence in enumerate(example_sentences):
            sentence_content, tag, meaning = example_sentence_pattern.findall(sentence)[0]
            sentence_content = header_pattern.sub("", sentence_content)
            sentence_content = f"{get_japanese_sequence_sign(sentence_seq+1)} {sentence_content}"

            sentence_content = transform_content_to_ruby_rich_text(sentence_content)
            formated_exmpale_sentences.append({"content": sentence_content, "tag": tag, "meaning": meaning})

        # example_sentences = [{"content": sentence} for sentence in map(transform_content_to_ruby_rich_text, detail.get("例文") or [])]

        formatted_grammars.append({
            "id": idx + 1,
            "content": content,
            "hiragana": hiragana,
            "meaning": detail.get("说明", "").strip(),
            "usage": detail.get("接续", "").strip(),
            "example": formated_exmpale_sentences,
            "remark": detail.get("注意", "").strip(),
            "source": "S2N1-N5",
            "japanese_meaning": "",
            "chinese_meaning": detail.get("说明", "").strip(),
            "level": detail["level"]
        })
    except Exception:
        print(grammar)
        print(content)
        print(detail)
        raise

print(formatted_grammars[0])
with open("grammars_N1-N5.json", "w", encoding="utf-8") as fp:
    fp.write(json.dumps(formatted_grammars, ensure_ascii=False, indent=4))

In [ ]:
# 日语平假名unicode编码范围：\u3040-\u309F
# 日语片假名unicode编码范围：\u30A0-\u30FF

In [ ]:
content = "私の料理を一口食べるなり 、父は変な顔をして席を立ってしまった。"
get_md5(content)

In [ ]:
import time
import os
from tqdm import tqdm
import re
import sys

pattern = re.compile(r"^[△\u2460-\u2470\s]*")
rich_text_pattern = re.compile(r"<ruby>(.*?)<rt>.*?</rt></ruby>")
RE_WHITESPACES_PATTERN = re.compile(r"\s+")

with open("grammars_N1-N5.json", "r", encoding="utf-8") as fp:
    formatted_grammars = json.loads(fp.read())

total_sentence_amount = sum([len(grammar["example"]) for grammar in formatted_grammars])
print(f"Total sentence amount: {total_sentence_amount}")

for grammar in tqdm(formatted_grammars, file=sys.stdout):
    for sentence in grammar["example"]:
        content = sentence["content"]
        content = pattern.sub("", content)
        content = rich_text_pattern.sub(r"\1", content)
        content = RE_WHITESPACES_PATTERN.sub("", content)

        content_md5 = get_md5(content)
        audio_path = f"japanese_grammar/audio/{content_md5}.mp3"

        text_to_audio.generate_audio([
            (None, content),
        ], audio_path, log=tqdm.write)
        time.sleep(1)


In [ ]:
grammar_content_pattern = re.compile(r"^\d? ?[~\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF  ()]+")

grammar_content_pattern.findall("~ 際 (に)")

In [ ]:
content = "ここに車を止められるのは、許可をもらっている人（a だけ　b に限り）です。"
content = re.sub("\s+", " ", content)
print(content)

question_pattern = re.compile(r"[（(](?:[\da-n]\s*[~\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF]+\s*)+[)）]")
print(question_pattern.findall(content))

In [ ]:
re.findall(r"(?:[\da-n]\s*[~\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF]+\s*)+", "a だけ b に限り")

In [ ]:
question_pattern = re.compile(r"[(（](?:[\da-n]\s*[~\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF]+\s*)+[）)]*\?")
print(question_pattern.findall("こちらの会議室をご利用になる際は、 受付で必要事項をご記入ください。"))
print(question_pattern.findall("ここに車を止められるのは、許可をもらっている人（a だけ　b に限り）です。"))

In [ ]:
import pandas as pd
import re

grammar_content_pattern = re.compile(r"^\d [~\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF  ()]+")
sentence_content_pattern = re.compile(r'([\u2460-\u2470])\s*')
invalid_line_pattern = re.compile(r'^[\d\s]+$')

example_sentences = []
with open("CS_Word_20240403_23.41.17.txt", "r", encoding="utf-8") as fp:
    matched = False
    sentence = None
    last_line = ""
    for line in fp.readlines():
        line = line.strip()
        if len(line) < 3 or invalid_line_pattern.match(line.strip()):
            continue

        if grammar_content_pattern.match(line):
            print(line)